In [11]:
import numpy as np
from deel.lip.activations import GroupSort2
from deel.lip.layers import (
    FrobeniusDense,
    ScaledL2NormPooling2D,
    SpectralConv2D,
    SpectralDense,
)
from deel.lip.losses import MulticlassHKR, MulticlassKR
from deel.lip.model import Sequential
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

import tensorflow as tf
import tensorflow.keras.backend as K

In [2]:
layers = [
    SpectralDense(100, kernel_initializer="orthogonal"),
    GroupSort2(),
    SpectralDense(100, kernel_initializer="orthogonal"),
    GroupSort2(),
    SpectralDense(32, kernel_initializer="orthogonal"),
    GroupSort2(),
    FrobeniusDense(10, activation=None, use_bias=False, kernel_initializer="orthogonal"),
]

model = Sequential(
    layers,
    k_coef_lip=1.0,
    name="hkr_model",
)

2023-04-18 17:06:30.296506: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
model.compile(
    # decreasing alpha and increasing min_margin improve robustness (at the cost of accuracy)
    # note also in the case of lipschitz networks, more robustness require more parameters.
    loss=MulticlassHKR(alpha=50, min_margin=0.05),
    optimizer=Adam(1e-3),
    metrics=["accuracy", MulticlassKR()],
)


# load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# standardize and reshape the data
x_train = np.expand_dims(x_train, -1)
mean = x_train.mean()
std = x_train.std()
x_train = (x_train - mean) / std
x_test = np.expand_dims(x_test, -1)
x_test = (x_test - mean) / std
# one hot encode the labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_test, (-1, 784))

In [4]:
# fit the model
model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=10,
    validation_data=(x_test, y_test),
    shuffle=True,
    verbose=0,
)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [5]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 0s 1ms/step - loss: -9.4709 - accuracy: 0.9698 - MulticlassKR: 11.2979


[-9.470858573913574, 0.9697999954223633, 11.297926902770996]

In [6]:
# once training is finished you can convert
# SpectralDense layers into Dense layers and SpectralConv2D into Conv2D
# which optimize performance for inference
vanilla_model = model.vanilla_export()

In [7]:
vanilla_model.summary()

Model: "hkr_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 spectral_dense (Dense)      (None, 100)               78500     
                                                                 
 group_sort2 (GroupSort2)    (None, 100)               0         
                                                                 
 spectral_dense_1 (Dense)    (None, 100)               10100     
                                                                 
 group_sort2_1 (GroupSort2)  (None, 100)               0         
                                                                 
 spectral_dense_2 (Dense)    (None, 32)                3232      
                                                                 
 group_sort2_2 (GroupSort2)  (None, 32)                0         
                                                                 
 frobenius_dense (Dense)     (None, 10)                32

In [12]:
from decomon import get_adv_box
from decomon.models import clone

In [27]:
C = Input((10, 10))
convex_domain = {"name": "ball", "p": 2, "eps": tf.Variable(0.)}

In [39]:
decomon_model_adv = clone(vanilla_model, back_bounds=[C])

C
E
C
E
C
E


In [61]:
x_min = x_test[:1]-eps_/1000
x_max = x_test[:1]+eps_/1000

In [62]:
box = np.concatenate([x_min[:,None], x_max[:,None]], 1)

In [63]:
upper = decomon_model_adv.predict([box, C_i])

1/1 [==============================] - 0s 37ms/step


In [68]:
x_test

array([[-0.42407389, -0.42407389, -0.42407389, ..., -0.42407389,
        -0.42407389, -0.42407389],
       [-0.42407389, -0.42407389, -0.42407389, ..., -0.42407389,
        -0.42407389, -0.42407389],
       [-0.42407389, -0.42407389, -0.42407389, ..., -0.42407389,
        -0.42407389, -0.42407389],
       ...,
       [-0.42407389, -0.42407389, -0.42407389, ..., -0.42407389,
        -0.42407389, -0.42407389],
       [-0.42407389, -0.42407389, -0.42407389, ..., -0.42407389,
        -0.42407389, -0.42407389],
       [-0.42407389, -0.42407389, -0.42407389, ..., -0.42407389,
        -0.42407389, -0.42407389]])

In [64]:
upper

[array([[-0.17667532, -2.7912064 , -1.2941009 , -2.0105286 , -2.439334  ,
         -0.26534486, -3.8664474 ,  0.        , -0.17121363,  4.429975  ]],
       dtype=float32),
 array([[-2.1759222, -5.5280433, -2.788595 , -3.753439 , -3.2562768,
         -1.9425571, -5.514657 ,  0.       , -1.911341 ,  1.2450018]],
       dtype=float32)]

retrive one radis bound

In [18]:
vanilla_model.predict(x_test[:1]).argmax()

1/1 [==============================] - 0s 19ms/step


7

In [20]:
y_test[:1].argmax()

7

In [24]:
tmp = vanilla_model.predict(x_test[:1])[0]
tmp.sort()
eps_=(tmp[-1]-tmp[-2])/np.sqrt(2)

1/1 [==============================] - 0s 20ms/step


In [35]:
K.set_value(convex_domain['eps'], eps_)

In [31]:
C_ = np.diag([1.0] * 10)[None] - y_test[:, :, None]
C_i = C_[:1]

In [32]:
C_i.shape

(1, 10, 10)

In [36]:
decomon_model_adv.predict(x_test[:1])

ValueError: in user code:

    File "/Users/ducoffe/miniconda3/envs/decomon_dev/lib/python3.9/site-packages/keras/engine/training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "/Users/ducoffe/miniconda3/envs/decomon_dev/lib/python3.9/site-packages/keras/engine/training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/ducoffe/miniconda3/envs/decomon_dev/lib/python3.9/site-packages/keras/engine/training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/ducoffe/miniconda3/envs/decomon_dev/lib/python3.9/site-packages/keras/engine/training.py", line 2079, in predict_step
        return self(x, training=False)
    File "/Users/ducoffe/miniconda3/envs/decomon_dev/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/ducoffe/min

In [9]:
decomon_model = clone(vanilla_model, method="crown")

C
E
C
E
C
E
